<a href="https://colab.research.google.com/github/leejunho12316/HonGongMachine/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0_%EC%8B%A4%EA%B8%B0_Part2_Chapter06_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
import pandas as pd
train = pd.read_csv('hr_train.csv')
test = pd.read_csv('hr_test.csv')

#EDA

In [5]:
print("shape:", train.shape, test.shape)

shape: (15326, 14) (3832, 13)


In [9]:
display("null : ",train.isnull().sum())
print()
display("null : ",test.isnull().sum())

'null : '

,0
enrollee_id,0
city,0
city_development_index,0
gender,3576
relevent_experience,0
enrolled_university,314
education_level,365
major_discipline,2281
experience,54
company_size,4787


'null : '

,0
enrollee_id,0
city,0
city_development_index,0
gender,932
relevent_experience,0
enrolled_university,72
education_level,95
major_discipline,532
experience,11
company_size,1151


In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15326 entries, 0 to 15325
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             15326 non-null  int64  
 1   city                    15326 non-null  object 
 2   city_development_index  15326 non-null  float64
 3   gender                  11750 non-null  object 
 4   relevent_experience     15326 non-null  object 
 5   enrolled_university     15012 non-null  object 
 6   education_level         14961 non-null  object 
 7   major_discipline        13045 non-null  object 
 8   experience              15272 non-null  object 
 9   company_size            10539 non-null  object 
 10  company_type            10383 non-null  object 
 11  last_new_job            14984 non-null  object 
 12  training_hours          15326 non-null  int64  
 13  target                  15326 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [2]:
train.head(3)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,30266,city_84,0.698,Male,No relevent experience,Full time course,High School,NaN,1,100-500,Pvt Ltd,never,15,0.0
1,13254,city_16,0.910,Male,Has relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,1,4,0.0
2,31675,city_21,0.624,Male,No relevent experience,Full time course,High School,NaN,3,NaN,NaN,never,12,0.0


In [3]:
test.head(3)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,23044,city_136,0.897,Female,No relevent experience,Full time course,Masters,STEM,8,100-500,Public Sector,1,35
1,29611,city_21,0.624,NaN,Has relevent experience,no_enrollment,Graduate,STEM,16,<10,Pvt Ltd,>4,134
2,22483,city_21,0.624,NaN,Has relevent experience,no_enrollment,Graduate,STEM,5,10000+,Pvt Ltd,1,92


#데이터 전처리

In [74]:
#타겟 데이터 나누기
target = train.pop('target')

In [75]:
#결측치 처리 (추가1)
#object 칼럼만 결측치 있으니 'X'로 교체
train.fillna('X', inplace=True)
test.fillna('X', inplace=True)

In [63]:
'''#데이터 합쳐서 한번에 OH Encoding
print(train.shape, test.shape)
data = pd.concat([train, test], axis=0)
print(data.shape)

from sklearn.preprocessing import OneHotEncoder
oh = OneHotEncoder()
data = oh.fit_transform(data).toarray()

train = data[:len(train), :]
test = data[len(train):, :]

print(train.shape, test.shape)'''

'#데이터 합쳐서 한번에 OH Encoding\nprint(train.shape, test.shape)\ndata = pd.concat([train, test], axis=0)\nprint(data.shape)\n\nfrom sklearn.preprocessing import OneHotEncoder\noh = OneHotEncoder()\ndata = oh.fit_transform(data).toarray()\n\ntrain = data[:len(train), :]\ntest = data[len(train):, :]\n\nprint(train.shape, test.shape)'

In [76]:
#데이터 합치기
print(train.shape, test.shape)
data = pd.concat([train, test], axis=0)
print(data.shape)

#data.drop('enrollee_id', axis=1, inplace=True) -> id삭제 성능저하

#Obj: Label Encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

data_columns = data.select_dtypes(include='object').columns
for col in data_columns:
  data[col] = le.fit_transform(data[col])

train = data.iloc[:len(train), :]
test = data.iloc[len(train):, :]
print(train.shape, test.shape)


#number : Scaling
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

(15326, 13) (3832, 13)
(19158, 13)
(15326, 13) (3832, 13)


#모델 학습

In [77]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.3, random_state = 99)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(10728, 13) (4598, 13) (10728,) (4598,)


In [92]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 300, max_depth = 50)
rfc.fit(X_train, y_train)

from sklearn.metrics import roc_auc_score
y_pred = rfc.predict_proba(X_val)

score = roc_auc_score(y_val, y_pred[:, 1])
print(score)

0.7962386013517266


#전처리
초기 : 0.7973004765952657

초기 + 결측치 처리 : 0.7953052615620428

초기 + 결측치 처리 + Label Encoding : 0.7953623039232215

초기 + 결측치 처리 + Label Encoding + RobustScaler: 0.7919277199762602

(X) 초기 + 결측치 처리 + Label Encoding + RobustScaler + id삭제 : 0.7884982064614037

#하이퍼파라미터

n_estimators = 100 : 0.794293583596341

n_estimators = 300 : 0.7963597846790302
+ max_depth = 100 : 0.7980884217443452

n_estimators = 500 : 0.7975702235832388

n_estimators = 1000 : 0.7971124903249817


#저장

In [93]:
submit = rfc.predict_proba(test)
submit_df = pd.DataFrame({'pred' : submit[:,1]})
submit_df.to_csv('result.csv', index=False)

In [94]:
read_df = pd.read_csv('result.csv')
read_df

,pred
0,0.250000
1,0.480000
2,0.376667
3,0.110000
4,0.060000
...,...
3827,0.233333
3828,0.243333
3829,0.133333
3830,0.170000
